# オートエンコーダ

In [1]:
import sys
sys.path.append('/notebooks/MOD2NN')

In [9]:
import time
import sys
import os

from Faraday.two_dim.module.lib.layers import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.9.1
Python: 3.9.13 (main, May 23 2022, 22:01:06) 
[GCC 9.4.0]


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [4]:
#@title デフォルトのタイトル テキスト
wavelength = 633.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}
z = 0.7e-3 #@param {type:"number"}

In [10]:
x_resized_train = tf.image.resize(tf.expand_dims(x_train, -1), (100, 100))
x_resized_train = tf.keras.layers.Lambda(lambda x: x[:, :, :, 0])(x_resized_train)

x_resized_val = tf.image.resize(tf.expand_dims(x_val, -1), (100, 100))
x_resized_val = tf.keras.layers.Lambda(lambda x: x[:, :, :, 0])(x_resized_val)

2022-11-09 05:23:04.595584: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 164.49MiB (rounded to 172480000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-11-09 05:23:04.595732: W tensorflow/core/common_runtime/bfc_allocator.cc:491] <allocator contains no memory>


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [8]:
def create_model(**kwargs):
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    theta = np.pi/2.
    eta = 0.
    tf.random.set_seed(kwargs["seed"])
    x_resized = ImageResizing(shape)(inputs)
    x = IntensityToElectricField(shape)(x_resized)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = PhotoMask(shape, row=3, col=3, width=10, height=10)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = ElectricFieldToIntensity(shape)(x)
    model = tf.keras.Model(inputs, x)
    return model

In [ ]:
model_name = "20221107_1"
cholab_path = "/content/drive/MyDrive/D2NN/"
checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
path = cholab_path + "trained_model/"+ model_name

model = create_model(seed=1)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.1),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # category: sparse_categorical_crossentropy
              metrics=[tf.keras.metrics.BinaryCrossentropy(from_logits=True)]
              )

epochs = 50
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        min_delta=0.0002,
        patience=2,
    )

def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  start = 0.2
  end = 0.0001
  n = 50
  a = -1/(n-1) * np.log(end/start)
  x = np.arange(n)
  return np.exp(-a*(epoch-1)) * start
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

logdir = os.path.join(cholab_path +"logs", model_name)
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

result = model.fit(x_train,
                   x_train,
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(x_val, x_resized_val),
                   callbacks=[lr_callback, cp_callback, tensorboard_callback, early_stopping]
                   )

model.save(path)

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")



with open(path + "/config.json", 'w') as f:
    json.dump(model.get_config(), f, indent=4)